In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision.models import resnet18
from torchvision import transforms
from tqdm import tqdm
from torch import nn
from torch.utils.data import Dataset,TensorDataset,DataLoader,random_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from d2l import torch as d2l
import random
import os
from torchprofile import profile_macs
import torch.nn.functional as F
from torch.autograd import grad
import torch.nn.init as init
d2l.use_svg_display()
from fvcore.nn import FlopCountAnalysis, flop_count_table
from models import models 
from models.models import get_data,train,calculate_flops_torchprofile,evaluation,calculate_mae,mape,calculate_model_size

In [2]:
from models.models import  TPA_CNN_LSTM as md
now_model = md()
size=calculate_model_size(now_model)
input_shape = (1, 11)  # batch_size=1
# total_flops, flops_table = calculate_flops_fvcore(now_model, input_shape)
total_flops = calculate_flops_torchprofile(now_model, input_shape, device='cpu')
print(f"模型大小:{size}M")
print(f"总FLOPs (近似值): {total_flops/1000000:,}")

模型大小:0.6328201293945312M
总FLOPs (近似值): 34.75702284288


D:\python_env\anaconda3\envs\work\lib\site-packages\torchprofile\profile.py:22: UserWarning: No handlers found: "aten::unsqueeze". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
D:\python_env\anaconda3\envs\work\lib\site-packages\torchprofile\profile.py:22: UserWarning: No handlers found: "aten::permute". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
D:\python_env\anaconda3\envs\work\lib\site-packages\torchprofile\profile.py:22: UserWarning: No handlers found: "aten::lstm". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


In [3]:
tests=[]
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
for i in range(5):
    test_result=train(now_model,seed=i,device=device,last=1e-5)
    tests.append(test_result)
    rmse_all=0
num=0
for i in range(len(tests)):
    for j in range(len(tests[0])):
        real,pred=tests[i][j][0],tests[i][j][1].reshape(-1)
        rmse=evaluation(real,pred)
        rmse_all+=rmse
        num+=1
print("RMSE:",rmse_all/num)

training seed 0:



D:\python_env\anaconda3\envs\work\lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: 5 Train_RMSELoss: tensor(0.0286, device='cuda:0', grad_fn=<DivBackward0>) 

Epoch: 5 valid_RMSELoss: tensor(0.0342, device='cuda:0') 

Epoch: 10 Train_RMSELoss: tensor(0.0286, device='cuda:0', grad_fn=<DivBackward0>) 

Epoch: 10 valid_RMSELoss: tensor(0.0340, device='cuda:0') 

Epoch: 15 Train_RMSELoss: tensor(0.0310, device='cuda:0', grad_fn=<DivBackward0>) 

Epoch: 15 valid_RMSELoss: tensor(0.0350, device='cuda:0') 

Epoch: 20 Train_RMSELoss: tensor(0.0310, device='cuda:0', grad_fn=<DivBackward0>) 

Epoch: 20 valid_RMSELoss: tensor(0.0400, device='cuda:0') 

Epoch: 25 Train_RMSELoss: tensor(0.0296, device='cuda:0', grad_fn=<DivBackward0>) 

Epoch: 25 valid_RMSELoss: tensor(0.0359, device='cuda:0') 

Epoch: 30 Train_RMSELoss: tensor(0.0300, device='cuda:0', grad_fn=<DivBackward0>) 

Epoch: 30 valid_RMSELoss: tensor(0.0347, device='cuda:0') 

Epoch: 35 Train_RMSELoss: tensor(0.0276, device='cuda:0', grad_fn=<DivBackward0>) 

Epoch: 35 valid_RMSELoss: tensor(0.0342, device='cuda:

KeyboardInterrupt: 